In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import os, json
import numpy as np

if not os.getcwd().endswith("src"):
    os.chdir("src")
    print("jumping into src")

from utils.data.data_module import DataModule
from utils.data.testbench import TestBench
from recommender.run_pipeline import Models

if os.getcwd().endswith("src"):
    os.chdir("..")
    print("jumping out of src")

args = {
    "output_dir": "models/popularity",
    "dataset_config": "configs/datasets/id_dataset.json",
    "model_config": "configs/twotower/3_user_id_anime_id_reweighting.json",
    "model": "tower",
    "should_return_ids": True,
}
dataset_config = {}
model_config = {}
output_dir = args["output_dir"]
if args["dataset_config"]:
    with open(args["dataset_config"], "r") as f:
        dataset_config_2 = json.load(f)
        dataset_config.update(dataset_config_2)
if args["model_config"]:
    with open(args["model_config"], "r") as f:
        model_config = json.load(f)

os.makedirs(output_dir, exist_ok=True)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

jumping into src
jumping out of src


In [3]:
datamodule = DataModule(**dataset_config)
testbench = TestBench(
    datamodule, should_return_ids=args.get("should_return_ids", False)
)

Resetting Train to k=0 ...: 100%|██████████| 48669/48669 [00:13<00:00, 3547.13it/s]

Number of Users: 54077, Hash[:8]: 9f0cd3, Hash: 9f0cd3119bd9ee7279856737c33aebb8
Total Animes: 12294, Total Users: 54077


In [20]:
auxiliary_args = {
    "n_users": datamodule.max_user_count,
    "n_anime": datamodule.max_anime_count,
}
model_config = model_config | auxiliary_args
model = Models.from_string(args["model"].upper())(datamodule=datamodule, **model_config)

model.train()

metrics = testbench.full_evaluation(model)
with open(os.path.join(output_dir, "output.txt"), "w") as f:
    for k, v in metrics.items():
        if type(v) == np.ndarray:
            continue
        f.write(f"{k}: {v}\n")

Epoch 1 / 20
[0] Loss: 0.6981. Time: 15.25 s / 305.06 s. ETA: 289.81 s
Start Time: 2024-11-13 16:52:42
End Time: 2024-11-13 16:52:44
This model took 1.9063 seconds.
Out of an optimal score of 1.0, you scored 0.0009.
Your DEI score is 87397.9234.
Your Pseudo-IOU score is 0.2172.
>>0|0.6984124183654785|{"runtime": 1.9063220024108887, "ndcg": 0.0009443532173076013, "diversity_score": 87397.92341426293, "pseudo_iou": 0.2171782544378698}<<
Epoch 2 / 20
[1] Loss: 0.6974. Time: 35.59 s / 355.90 s. ETA: 320.31 s
Start Time: 2024-11-13 16:53:03
End Time: 2024-11-13 16:53:05
This model took 1.9068 seconds.
Out of an optimal score of 1.0, you scored 0.0011.
Your DEI score is 87348.9405.
Your Pseudo-IOU score is 0.2169.
>>1|0.6964989304542542|{"runtime": 1.9067797660827637, "ndcg": 0.0010525868991938133, "diversity_score": 87348.94049619531, "pseudo_iou": 0.21688239644970414}<<
Epoch 3 / 20
[2] Loss: 0.6925. Time: 56.49 s / 376.58 s. ETA: 320.09 s
Start Time: 2024-11-13 16:53:24
End Time: 2024-11-

KeyboardInterrupt: 